In [8]:
import pandas as pd

df = pd.read_csv('./3/ready_dataset3.csv', delimiter=';', encoding='utf-8')
df.head()

,text,from,to,moment
0,Depuis La Chaize le vicomte à La Roche sur foron,La Chaize le vicomte,La Roche sur foron,NaN
1,Itiniréraire jusqu'a Giromagny depuis Quimper,Quimper,Giromagny,NaN
2,Comment aller de Boigneville à Longjumeau mardi,Boigneville,Longjumeau,mardi
3,Je suis actuellement à Villers sur mer et j ai...,Villers sur mer,Ferrières en bray,NaN
4,Donne moi l'itinéraire pour aller à Fontenay l...,Lizy sur ourcq,Fontenay le fleury,NaN


In [9]:
len(df)

6565

In [10]:
# remove duplicates
df = df.drop_duplicates()
len(df)

6564

In [96]:
# remove punctuations
import re

def process_text(text):
    if(pd.isna(text)):
        return text
    cleaned_text = re.sub(r'[^a-zA-Z0-9À-ÖØ-öø-ÿ\' ]', ' ', text)  # Remove punctuation
    return cleaned_text

# Apply the function to the 'text' column
df['text'] = df['text'].apply(process_text)
df['from'] = df['from'].apply(process_text)
df['to'] = df['to'].apply(process_text)

df.head()

,text,from,to,moment
0,Depuis La Chaize le vicomte à La Roche sur foron,La Chaize le vicomte,La Roche sur foron,NaN
1,Itiniréraire jusqu'a Giromagny depuis Quimper,Quimper,Giromagny,NaN
2,Comment aller de Boigneville à Longjumeau mardi,Boigneville,Longjumeau,mardi
3,Je suis actuellement à Villers sur mer et j ai...,Villers sur mer,Ferrières en bray,NaN
4,Donne moi l'itinéraire pour aller à Fontenay l...,Lizy sur ourcq,Fontenay le fleury,NaN


In [97]:
# Remove sentences with from or to misspelled
import re

def check_cities(row):
    valid_from = True
    valid_to = True
    
    if pd.notna(row['from']):
        valid_from = row['from'] in row['text']
    
    if pd.notna(row['to']):
        valid_to = row['to'] in row['text']
    
    return valid_from and valid_to

# Use the function to create a boolean mask
mask = df.apply(check_cities, axis=1)

# Filter the DataFrame using the mask
df = df[mask]

len(df)

3565

In [98]:
df.head()

,text,from,to,moment
0,Depuis La Chaize le vicomte à La Roche sur foron,La Chaize le vicomte,La Roche sur foron,NaN
1,Itiniréraire jusqu'a Giromagny depuis Quimper,Quimper,Giromagny,NaN
2,Comment aller de Boigneville à Longjumeau mardi,Boigneville,Longjumeau,mardi
3,Je suis actuellement à Villers sur mer et j ai...,Villers sur mer,Ferrières en bray,NaN
4,Donne moi l'itinéraire pour aller à Fontenay l...,Lizy sur ourcq,Fontenay le fleury,NaN


In [ ]:
# Select lines which contains brackets
df[df['text'].str.contains(r'\(.*\)', na=False)]

In [116]:
import random

def generate_sentences(k, percentage_from_not_na, percentage_to_not_na):
    # Filter rows where both 'FROM' and 'TO' are not NaN
    filtered_df = df.dropna(subset=['from', 'to'])

    replacement_cities = pd.read_csv('./cities.csv')['city_code'].to_list()

    # Randomly select rows and replace 'FROM' and 'TO' with random cities
    random_rows = random.sample(range(len(filtered_df)), k=k)  # Replace 2 with the number of random rows you want to select

    # Create a list to store the modified rows
    modified_rows = []

    for idx in random_rows:
        new_from = random.choice(replacement_cities) if random.random() < percentage_from_not_na else None  # chance of setting as NaN
        new_to = random.choice(replacement_cities) if random.random() < percentage_to_not_na else None  # chance of setting as NaN
        modified_row = filtered_df.iloc[idx].copy()  # Create a copy of the selected row
        
        # Replace 'FROM' and 'TO' cities within the 'text' with new values
        modified_row['text'] = modified_row['text'].replace(modified_row['from'], new_from if new_from else "")
        modified_row['text'] = modified_row['text'].replace(modified_row['to'], new_to if new_to else "")

        # Update 'from' and 'to' columns with new values
        modified_row['from'] = new_from
        modified_row['to'] = new_to
        
        modified_rows.append(modified_row)

    # Create a new DataFrame with the modified rows
    new_sentences_df = pd.DataFrame(modified_rows)
    return new_sentences_df



In [118]:
A = generate_sentences(1000, 1, 1)
B = generate_sentences(1000, 1, 0)
C = generate_sentences(1000, 0, 1)

In [120]:
C

,text,from,to,moment
290,itinéraire jusqu'a ban de laveline en partant de,None,ban de laveline,NaN
1460,jeudi après midi je partirais de pour aller à...,None,st sauveur,jeudi après-midi
1836,La ligne moulin mage,None,moulin mage,NaN
654,Je me rends à taillebourg en partant de,None,taillebourg,NaN
1253,J'irai bien à la gare de dampierre les bois de...,None,dampierre les bois,NaN
...,...,...,...,...
3899,J'aurais besoin de directions pour aller de à...,None,ottonville,NaN
1850,Je cherche un train vendome,None,vendome,NaN
1796,Je cherche un train le manoir,None,le manoir,NaN
3481,Comment rejoindre depuis fressines,None,fressines,NaN


In [121]:
# Concatenate the new DataFrame with the original DataFrame
df = pd.concat([df, A], ignore_index=True)
df = pd.concat([df, B], ignore_index=True)
df = pd.concat([df, C], ignore_index=True)

In [123]:
# Count lines where both 'from' and 'to' are NaN
both_nan_count = len(df[(df['from'].isna()) & (df['to'].isna())])

# Count lines where 'from' contains a value and 'to' is NaN
from_not_nan_count = len(df[(~df['from'].isna()) & (df['to'].isna())])

# Count lines where 'from' is NaN and 'to' contains a value
to_not_nan_count = len(df[(df['from'].isna()) & (~df['to'].isna())])

# Count lines where both 'from' and 'to' contain values
both_not_nan_count = len(df[(~df['from'].isna()) & (~df['to'].isna())])

print(f'Number of sentences: {len(df)}')
print(f"Both 'from' and 'to' are NaN: {both_nan_count} lines")
print(f"'from' has a value and 'to' is NaN: {from_not_nan_count} lines")
print(f"'from' is NaN and 'to' has a value: {to_not_nan_count} lines")
print(f"Both 'from' and 'to' have values: {both_not_nan_count} lines")

Number of sentences: 6565
Both 'from' and 'to' are NaN: 923 lines
'from' has a value and 'to' is NaN: 1098 lines
'from' is NaN and 'to' has a value: 1269 lines
Both 'from' and 'to' have values: 3275 lines


In [125]:
df.to_csv('./cleaned_dataset3.csv', sep=";", index=False)